In [ ]:
import scrapy

In [ ]:
class Spider12(scrapy.Spider):
    name = 'spider12'
    # dominios a scrapear
    allowed_domains = ['pagina12.com.ar']  
    # formato de archivo de salida
    custom_settings = {'FEED_FORMAT':'json',   
                       'FEED_URI': 'resultados.json',
                       'DEPTH_LIMIT': 2}  
    
    # URLS a scrapear
    starts_urls = ['https://www.pagina12.com.ar/secciones/el-pais',
                  'https://www.pagina12.com.ar/secciones/economia',
                  'https://www.pagina12.com.ar/secciones/sociedad',
                  'https://www.pagina12.com.ar/suplementos/cultura-y-espectaculos',
                  'https://www.pagina12.com.ar/secciones/el-mundo',
                  'https://www.pagina12.com.ar/secciones/deportes',
                  'https://www.pagina12.com.ar/secciones/contratapa',
                  'https://www.pagina12.com.ar/secciones/audiovisuales']
    
    # Procesar la respuesta de cada solicitud
    def parse(self, response):
        
        # Articulo promocionado
        nota_promocionada = response.xpath('//div[@class="featured-article__container"]/h2/a/@href').get()
        if nota_promocionada is not None:
            # Pasar la respuesta a parse_nota
            yield response.follow(nota_promocionada, callback=self.parse_nota)
        
        # Listado de notas
        notas = response.path('//ul[@class="article-list"]//li//a/@href').getall()
        for nota in notas: 
            # Pasar la respuesta a parse_nota
            yield response.follow(nota, callback=self.parse_nota)
        
        
    def parse_nota(self, response):
        date = response.xpath('//div[@class="time"]/span/@datetime').get()
        prefix = response.xpath('//h2[@class="article-prefix"]/text()').get()
        title = response.xpath('//h1[@class="article-titles"]/text()').get()
        summary = response.xpath('//div[@class="article-summary"]/text()').get()
        content = response.xpath('//div[@class="article-text"]/text()').getall()
        image = response.xpath('//div[@class="article-main-media-image__container"]/img/@src').getall()[-1] 
        yield {'url': response.url,
                'title':title,
                'cuerpo':cuerpo,
                }

In [ ]:
from scrapy.crawler import CrawlerProcess

In [ ]:
process = CrawlerProcess()
process.crawl(Spider12)
process.start()